In [ ]:
file_path = 'C:/Users/minkyu/Desktop/open/건설안전지침/'

In [ ]:
import pymupdf4llm
llama_reader = pymupdf4llm.LlamaMarkdownReader()


In [ ]:
import re
import textwrap


def remove_special_chars_and_extra_spaces(text):
    """
    연속적으로 사용되는 특수문자(예: --- 또는 ~~~)를 제거하고,
    1칸 이상의 연속된 띄어쓰기를 단일 공백으로 변경한다.
    """
    # '-' 또는 '~' 문자가 3회 이상 연속될 경우 제거
    text = re.sub(r'([-~]){3,}', '', text)
    # 2칸 이상의 공백을 단일 공백으로 변경
    text = re.sub(r'\s{2,}', ' ', text)
    return text

def remove_markdown_artifacts(text):
    """불필요한 markdown 문법(예: 헤더, 리스트 기호 등)을 제거"""
    # 예시: '#' 로 시작하는 헤더 제거
    text = re.sub(r'^#+\s*', '', text, flags=re.MULTILINE)
    # 예시: 리스트 기호('-','*') 제거
    text = re.sub(r'^[\-\*]\s+', '', text, flags=re.MULTILINE)
    return text

def reflow_text(text, width=80):
    """문단별로 텍스트를 리플로우하여 일정 너비로 맞춤"""
    paragraphs = text.split('\n')
    new_paragraphs = [textwrap.fill(p, width=width) for p in paragraphs if p.strip() != ""]
    return '\n\n'.join(new_paragraphs)

def clean_extracted_markdown(text, width=80):
    # 2. 특수문자 및 공백 정리
    text = remove_special_chars_and_extra_spaces(text)
    # 3. 마크다운 아티팩트 제거
    text = remove_markdown_artifacts(text)
    # 4. 리플로우로 가독성 향상
    text = reflow_text(text, width=width)
    return text

# 사용 예제:
# md_text = pymupdf4llm.to_markdown(file_path + "갱폼(Gang form) 제작 및 사용안전 지침.pdf")
# cleaned_text = clean_extracted_markdown(md_text)
# print(cleaned_text)


In [ ]:
md_text = pymupdf4llm.to_markdown(file_path+ "시스템 비계 안전작업 지침.pdf")

In [ ]:
cleaned_text = clean_extracted_markdown(md_text)
print(cleaned_text)

In [ ]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
text_splitter = SemanticChunker(embeddings)

In [ ]:
chunks = text_splitter.split_text(cleaned_text)

In [ ]:
import textwrap

# 예를 들어, width를 80으로 설정하여 감싸기
wrapped_text = textwrap.fill(chunks[1], width=80)
print(wrapped_text)

In [ ]:
import os
import re
import textwrap
import pymupdf4llm
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS

# 전처리 함수들
def remove_newlines_except_after_period(text):
    """마침표 다음의 줄바꿈을 제외한 모든 줄바꿈을 제거"""
    return re.sub(r'(?<!\.)(\n|\r\n)', ' ', text)

def remove_special_chars_and_extra_spaces(text):
    """
    연속적으로 사용되는 특수문자(예: --- 또는 ~~~)를 제거하고,
    1칸 이상의 연속된 띄어쓰기를 단일 공백으로 변경한다.
    """
    text = re.sub(r'([-~]){3,}', '', text)
    text = re.sub(r'\s{2,}', ' ', text)
    return text

def remove_markdown_artifacts(text):
    """불필요한 markdown 문법(예: 헤더, 리스트 기호 등)을 제거"""
    text = re.sub(r'^#+\s*', '', text, flags=re.MULTILINE)
    text = re.sub(r'^[\-\*]\s+', '', text, flags=re.MULTILINE)
    return text

def reflow_text(text, width=80):
    """문단별로 텍스트를 리플로우하여 일정 너비로 맞춤"""
    paragraphs = text.split('\n')
    new_paragraphs = [textwrap.fill(p, width=width) for p in paragraphs if p.strip() != ""]
    return '\n\n'.join(new_paragraphs)

def clean_extracted_markdown(text, width=80):
    # 1. 줄바꿈 정리
    text = remove_newlines_except_after_period(text)
    # 2. 특수문자 및 공백 정리
    text = remove_special_chars_and_extra_spaces(text)
    # 3. 마크다운 아티팩트 제거
    text = remove_markdown_artifacts(text)
    # 4. 리플로우로 가독성 향상
    text = reflow_text(text, width=width)
    return text

# PDF 파일이 있는 폴더 경로
file_path = 'C:/Users/minkyu/Desktop/open/건설안전지침/'

# Hugging Face 임베딩 및 SemanticChunker 초기화
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
text_splitter = SemanticChunker(embeddings)

# 벡터 스토어에 추가할 문서(Document)들을 저장할 리스트
documents = []

# 폴더 내의 모든 PDF 파일에 대해 처리
for filename in os.listdir(file_path):
    if filename.lower().endswith('.pdf'):
        full_path = os.path.join(file_path, filename)
        # PDF를 Markdown 형태의 텍스트로 변환 (pymupdf4llm 사용)
        md_text = pymupdf4llm.to_markdown(full_path)
        # 전처리: 불필요한 줄바꿈, 특수문자, markdown 아티팩트 제거 및 리플로우
        cleaned_text = clean_extracted_markdown(md_text)
        # 의미론적 청킹: 텍스트를 의미 단위의 청크로 분할
        chunks = text_splitter.split_text(cleaned_text)
        
        # 각 청크를 Document 객체로 변환하여 메타데이터(원본 파일명) 추가
        for chunk in chunks:
            documents.append(Document(page_content=chunk, metadata={"source": filename}))

# FAISS 벡터 스토어 생성: 문서들을 임베딩하고 인덱싱
vector_store = FAISS.from_documents(documents, embeddings)

# 이제 vector_store를 활용해 RAG 시스템에 질의 응답을 진행할 수 있습니다.
print(f"{len(documents)}개의 문서 청크가 벡터 스토어에 저장되었습니다.")


In [ ]:
vector_store.save_local("faiss_index")

In [ ]:
# 저장된 벡터 스토어 불러오기
vector_store.save_local("faiss_index")

# 불러오기 (같은 임베딩 객체를 사용)
from langchain.vectorstores import FAISS
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
new_vector_store = FAISS.load_local("faiss_index", embeddings)